In [1]:
import os
print(os.getcwd())

os.chdir("../")
print(os.getcwd())

/home/kumar/Documents/NEW JOB/UPSKILL/10 PERSONAL PROJECTS/RENAL-TUMOR-CLASSIFICATION/research
/home/kumar/Documents/NEW JOB/UPSKILL/10 PERSONAL PROJECTS/RENAL-TUMOR-CLASSIFICATION


In [7]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/tinytachyon14341/RENAL-DISEASE-CLASSIFIER-MLFLOW-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "tinytachyon14341"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "f7f82fe5075a413434f0e5cd779d34f62dd5fd9d"

In [8]:
%env MLFLOW_TRACKING_URI=https://dagshub.com/tinytachyon14341/RENAL-DISEASE-CLASSIFIER-MLFLOW-DVC.mlflow
%env MLFLOW_TRACKING_USERNAME=tinytachyon14341
%env MLFLOW_TRACKING_PASSWORD=f7f82fe5075a413434f0e5cd779d34f62dd5fd9d

env: MLFLOW_TRACKING_URI=https://dagshub.com/tinytachyon14341/RENAL-DISEASE-CLASSIFIER-MLFLOW-DVC.mlflow
env: MLFLOW_TRACKING_USERNAME=tinytachyon14341
env: MLFLOW_TRACKING_PASSWORD=f7f82fe5075a413434f0e5cd779d34f62dd5fd9d


In [9]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class EvaluteConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [11]:
from renal_tumor_detection.constants import *
from renal_tumor_detection.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH, 
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluteConfig:
        eval_config = EvaluteConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney_ct_data",
            mlflow_uri="https://dagshub.com/tinytachyon14341/RENAL-DISEASE-CLASSIFIER-MLFLOW-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE)
        return eval_config

In [12]:
import mlflow
import mlflow.keras
import tensorflow as tf
from pathlib import Path
from urllib.request import urlparse

class Evaluation:
    def __init__(self, config: EvaluteConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def laod_model(path: Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.laod_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="RenalDiseaseClassifier")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-07-26 23:41:27,748: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-26 23:41:27,751: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-26 23:41:27,752: INFO: common: created directory at: artifacts]
Found 119 images belonging to 2 classes.


2024-07-26 23:41:28.271093: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


8/8 [==============================] - 21s 3s/step - loss: 20.8347 - accuracy: 0.4958
[2024-07-26 23:41:49,393: INFO: common: json file saved at: scores.json]


2024/07/26 23:41:52 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-07-26 23:41:53,202: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: /tmp/tmpsa4sm9_u/model/data/model/assets
[2024-07-26 23:41:53,639: INFO: builder_impl: Assets written to: /tmp/tmpsa4sm9_u/model/data/model/assets]


Registered model 'RenalDiseaseClassifier' already exists. Creating a new version of this model...
2024/07/26 23:42:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: RenalDiseaseClassifier, version 5
Created version '5' of model 'RenalDiseaseClassifier'.
